In [1]:
from scipy.linalg import expm,eigh
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [2]:
L = 3 #system size
M = 0.1 #fermion mass
N=1 #number of layers
l=1 #spin length
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [3]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |2 2 2>           26  
      1.         |2 2 1>           25  
      2.         |2 2 0>           24  
      3.         |2 1 2>           23  
      4.         |2 1 1>           22  
      5.         |2 1 0>           21  
      6.         |2 0 2>           20  
      7.         |2 0 1>           19  
      8.         |2 0 0>           18  
      9.         |1 2 2>           17  
     10.         |1 2 1>           16  
     11.         |1 2 0>           15  
     12.         |1 1 2>           14  
     13.         |1 1 1>           13  
     14.         |1 1 0>           12  
     15.         |1 0 2>           11  
     16.         |1 0 1>           10  
     17.         |1 0 0>            9  
     18.         |0 2 2>            8  
     19.         |0 2 1>            7  
     20.         |0 2 0>            6  
     21.         |0 1 2>            5  
     22.         |0 1 1>            4  
     23.   

In [4]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]]
        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]]
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]]
        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))


In [5]:
#basis vectors as arrays
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [6]:
#projectors on g=1 for every site
Proj = [np.zeros((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for state in basis_vectors:
        state = np.array(state)
        if (Gauss_law[i].expt_value(state)>=1.-1e-8) & (Gauss_law[i].expt_value(state)<=1.+1e-8):
            Proj[i] += np.outer(state,state)

In [11]:
#initialisation of the unitarily transformed Hamiltonian

linear_term = []
quadratic_term = []
interaction_p = []
interaction_m = []


for i in range(L-1):
    linear_term.append([[0.5/np.sqrt(2),i]])
    quadratic_term.append([[(-1)**i/np.sqrt(2),i,i+1]])
    interaction_p.append([
        ["+z", quadratic_term[i]],
        ["+", linear_term[i]]
    ])
    interaction_m.append([
        ["-z", quadratic_term[i]],
        ["-", linear_term[i]]
    ])

linear_term.append([[0.5/np.sqrt(2),L-1]])
interaction_p.append([
    ["+", linear_term[L-1]]
])
interaction_m.append([
    ["-", linear_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [np.zeros((basis.Ns,basis.Ns)) for i in range(L)]
h_int = np.zeros((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].toarray()@Proj[i+1]+Proj[i+1]@H_int_m[i].toarray()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.toarray()

h_int = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints
#full_ham_matrix = sparse.coo_matrix(full_ham_matrix)

G=0
for i in range(L+1):
    G += 1e1*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.toarray()
print(full_ham_matrix)
constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term
#constrained_full_ham_matrix = sparse.coo_matrix(constrained_full_ham_matrix)

[[ 1.7+0.j  0. +0.j  0. +0.j -0.5+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  1. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j]
 [ 0. +0.j  0. +0.j  1.3+0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j]
 [-0.5+0.j  0. +0.j  0. +0.j  1.4+0.j  0.5+0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0.5+0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j  0. +0.j
   0. +0.j  0. +0.j  0

In [8]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
#eigenval, eigenvec = scipy.sparse.linalg.eigsh(full_ham_matrix,k=1,which = "SA")
eigenval, eigenvec = eigh(full_ham_matrix)

print(eigenval[eigenval<=1e1])

[-0.64551385  0.03967222  0.03967222  0.3         0.42748942  0.6
  0.6         0.6         0.7         0.9         0.976925    1.
  1.          1.          1.          1.26032778  1.26032778  1.3
  1.3         1.3         1.4         1.4         1.49754183  1.7
  1.7         2.1         2.24355759]


In [9]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
#eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(constrained_full_ham_matrix,k=1,which = "SA")
eigenvalues, eigenvectors = eigh(constrained_full_ham_matrix)

In [10]:
#Gauge invariant part of the spectrum
print(eigenvalues[eigenvalues<=1e1])
print(eigenvectors[:,0])


[-0.64551385  0.42748942  0.7         0.976925    1.49754183  2.24355759]
[ 3.97156170e-02-0.j  0.00000000e+00+0.j  1.11022302e-16+0.j
  1.86307059e-01+0.j -3.61235861e-01+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
 -3.61235861e-01+0.j  7.85788646e-01+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j -2.92003181e-01+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
  0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]


In [85]:
#initial state
string = ""
for i in range(L):
    string +="0"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.


In [86]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

81.3


In [87]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

(1.3+0j)


In [88]:
def clause(a,b,k,m,n):
    zero = 0
    m = int(m)
    n = int(n)
    for l in range(L):
        if l == k:
            if (int(a[l])!=m) | (int(b[l])!=n):
                zero +=1
        else:
            if a[l]!=b[l]:
                zero +=1
    return zero

In [89]:
def sigma(k,m,n,phi):
    sigma = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
        for j in range(0,i+1):
            string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)
            if (clause(string_i,string_j,k,m,n)==0):
                sigma[i][j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
                sigma[j][i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    return sigma
def ms(k,m,n,phi):
    return ((sigma(k,m,n,phi)+sigma(k+1,m,n,phi))**2)/2 

In [95]:
def sigma_g(m,n,phi):
    sigma_g = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(L):
        sigma_g +=sigma(i,m,n,phi)
        return sigma_g
    
def ms_g(m,n,phi):
    ms_g = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(L-1):
        ms_g += ((sigma(i,m,n,phi)+sigma(i+1,m,n,phi))**2)/2 
    return ms_g 

In [96]:
def rot(sigma,theta):
    return expm(-1j*theta*sigma/2)

In [97]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [112]:
#cost function definition
def cost_function_sigma(theta,psi,N):
    if len(theta)!=6*N+3:
        print("Warning: The number of variational parameters does not match the number of layers!")
    psi_var = psi
    for i in range(N):
        psi_var = rot(ms_g(0,2,0),theta[6*i+5])@rot(ms_g(1,2,0),theta[6*i+4])@rot(ms_g(0,1,0),theta[6*i+3])@rot(sigma_g(0,2,0),theta[6*i+2])@rot(sigma_g(1,2,0),theta[6*i+1])@rot(sigma_g(0,1,0),theta[6*i+0])@psi_var
    psi_var = rot(sigma_g(0,2,0),theta[6*N+2])@rot(sigma_g(1,2,0),theta[6*N+1])@rot(sigma_g(0,1,0),theta[6*N+0])@psi_var
    expt_value = 0
    matvec = constrained_full_ham_matrix@psi_var
    for i in range(basis.Ns):
        expt_value += np.conj(psi_var[i])*matvec[i]
    cost = expt_value
    return np.real(cost)

In [115]:
#optimisation algorithm
from scipy.optimize import dual_annealing
import datetime
duan_ranges = []
for i in range(6*N+3):
    duan_ranges.append((-0.1,2*np.pi+0.1))
resduan = 0
print(datetime.datetime.now())
resduan = dual_annealing(cost_function_sigma, duan_ranges, args = (psi_0,N))
print(datetime.datetime.now())

2022-02-26 20:12:35.977779
2022-02-26 20:25:09.736513


In [116]:
#results
print(resduan.fun)
print(resduan.x)


41.000000000025736
[-4.97268204e-06  3.86511003e+00  6.28318640e+00 -1.00155114e-06
  2.47617227e+00 -2.33933949e-07  3.12709573e-05  5.66414242e+00
 -4.57935402e-02]


In [111]:
#fidelity of the result
theta = resduan.x

psi_var = psi_0
for i in range(N):
    psi_var = rot(ms_g(0,2,0),theta[8*i+7])@rot(ms_g(1,2,0),theta[8*i+6])@rot(ms_g(0,1,0),theta[8*i+5])@rot(sigma_g(0,2,0),theta[8*i+4])@rot(sigma_g(1,2,theta[8*i+3]),theta[8*i+2])@rot(sigma_g(0,1,theta[8*i+1]),theta[8*i+0])@psi_var
psi_var = rot(sigma_g(0,2,0),theta[8*N+4])@rot(sigma_g(1,2,theta[8*N+3]),theta[8*N+2])@rot(sigma_g(0,1,theta[8*N+1]),theta[8*N+0])@psi_var
print(np.abs(np.dot(np.conj(psi_var),eigenvectors[:,0]))**2)
expt_value = 0
matvec = full_ham_matrix@psi_var
for i in range(basis.Ns):
    expt_value += np.conj(psi_var[i])*matvec[i]
print(np.real(expt_value))

IndexError: index 12 is out of bounds for axis 0 with size 9